In [1]:
import time
import numpy as np
import scipy as sp
import pandas as pd
import networkx as nx

## Arnoldi 

This is a copy of the algorithm defined and tested in the notebook `arnoldi.ipynb`. It's an implementation of the Algorithm 2 from the paper. It's needed during the computation of the shifted GMRES algorithm that we are trying to implement here.

In [2]:
def Arnoldi(A,v0,m):
    v = v0
    beta = np.linalg.norm(v)
    v = v/beta
    H = sp.sparse.lil_matrix((m+1,m)) 
    V = sp.sparse.lil_matrix((A.shape[0],m+1))
    V[:,0] = v # each column of V is a vector v

    for j in range(m):
        w = A @ v  
        for i in range(j):
            H[i,j] = v.T @ w # tmp is a 1x1 matrix, so it's O(1) in memory
            w = w - H[i,j]*v 
            
        H[j+1,j] = np.linalg.norm(w)

        if H[j+1,j] == 0:
            # print("Arnoldi breakdown")
            m = j
            v = 0
            break
        else:
            if j < m-1:
                v = w/H[j+1,j]
                V[:,j+1] = v

    return V, H, beta, j  

# Algorithm 4 testing

This algorithm is based on the "Algorithm 4" of the paper, the pseudocode provided by the authors is the following 

![](https://i.imgur.com/H92fru7.png)

Auxiliary function

In [3]:
def compute_gamma(res, a, k): # function to compute gamma
    gamma = np.ones(len(a))
    for i in range(len(a)):
        if i != k:
            gamma[i] = (res[i]*a[k])/(res[k]*a[i])

    return gamma

Auxiliary function to compute the $\tilde P$ matrix described in the reference paper

In [4]:
def google_matrix_sparse(G, alpha=0.85, personalization=None, nodelist=None, weight="weight", dangling=None) -> np.matrix:

  if nodelist is None:
      nodelist = list(G)

  A = nx.to_scipy_sparse_array(G, nodelist=nodelist, weight=weight, format="lil", dtype=int)

  N = len(G)
  if N == 0:
      return np.asmatrix(A)

  # Personalization vector
  if personalization is None:
      p = np.repeat(1.0 / N, N)
      p = sp.sparse.lil_array(p)
  else:
      p = np.array([personalization.get(n, 0) for n in nodelist], dtype=float)
      if p.sum() == 0:
          raise ZeroDivisionError
      p /= p.sum()

  # Dangling nodes
  if dangling is None:
      dangling_weights = np.ones(N, dtype=int)
      dangling_weights = sp.sparse.lil_array(dangling_weights, dtype=int)
  else:
      # Convert the dangling dictionary into an array in nodelist order
      dangling_weights = np.array([dangling.get(n, 0) for n in nodelist], dtype=float)
      dangling_weights /= dangling_weights.sum()

  # replace rows with all zeros with dangling_weights
  A[[A.sum(axis=1)==0],:] = dangling_weights

  # Normalize rows
  row_sums = A.sum(axis=1) # row sums
  r_inv = np.power(row_sums.astype(float), -1).flatten() # inverse of row sums
  r_inv[np.isinf(r_inv)] = 0.0 # replace inf with 0
  R = sp.sparse.diags(r_inv) # create diagonal matrix
  A = R.dot(A) 

  return A, p.T

`Algorithm 4` described in the reference paper 

In [5]:
def Algo4(Pt, v, m, a: list, tau, maxit: int, x):

    mv, iter = 0, 1 # mv is the number of matrix-vector products, iter is the number of iterations
    
    # initialize x as a random sparse matrix. Each col is the pagerank vector for a different alpha
    x = sp.sparse.lil_matrix((Pt.shape[0], len(a)))

    # initialize the identity matrix of size Pt.shape
    I = sp.sparse.eye(Pt.shape[0], Pt.shape[1], format='lil')

    # compute the residual vector, it is a matrix of size (n, len(a)). Each col is the residual vector for a different alpha. 
    r = sp.sparse.lil_matrix((Pt.shape[0], len(a)))
    res = np.zeros(len(a))

    # compute the residual vector and the norm of each col in the vector res
    for i in range(len(a)):
        r[:,[i]] = ((1 - a[i])/a[i])*v - ((1/a[i])*I - Pt).dot(x[:,[i]])
        res[i] = sp.sparse.linalg.norm(r[:,[i]], ord='fro') # frobenius norm since l2 norm is not defined for sparse matrices in scipy

    for _ in range(maxit):
        err = np.absolute(np.max(res)) # check if we have converged
        if err < tau:
            print("Computation ended successfully in ", iter, " iterations and ", mv, " matrix-vector products.")
            return x, iter, mv

        print("\niter = ", iter)
        print("res: ", res)
        print("err = ", err)

        # find k as the index of the largest residual
        k = int(np.argmax(res))
        print("k = ", k)

        # compute gamma as defined in the paper
        gamma = compute_gamma(res, a, k)
       
        # Run Arnoldi
        r_k = r[:,[k]].toarray() # r_k is the residual vector for alpha_k
        V, H, beta, j = Arnoldi((1/a[k])*I - Pt, r_k, m) # run Arnoldi
        H = H[:-1,:] # remove the last row of H
        V = V[:,:-1] # remove the last col of V
        mv = mv + j # update the number of matrix-vector products

        H_e1 = np.zeros(H.shape[0]) 
        H_e1[0] = 1 # canonical vector e1 of size H.shape[0]

        # compute y as the minimizer of || beta*e1 - Hy ||_2 using the least squares method
        y = sp.sparse.lil_matrix((H.shape[1],len(a))) 

        # we only need the k-th col of y in this iteration
        y[:,[k]] = sp.sparse.linalg.lsqr(H, beta*H_e1)[0]

        # # Update x
        x_new = x
        x_new[:,[k]] = x[:,[k]] + V @ y[:,[k]]

        V_e1 = np.zeros(V.shape[0])
        V_e1[0] = 1 
        V_e1 = sp.sparse.lil_matrix(V_e1) # canonical vector e1 of size V.shape[0]

        # Update res[k]
        res[k] = a[k]* (sp.sparse.linalg.norm(beta*V_e1.T - V @ y[:,[k]]))

        # multi shift
        for i in range(len(a)):
            if i != k and res[i] >= tau:
                if res[i] >= tau:
                    
                    H = H + ((1-a[i])/a[i] - (1-a[k])/a[k])*sp.sparse.eye(H.shape[0], H.shape[1], format='lil')

                    # Compute z as described in the paper
                    z1 = H_e1.T*beta
                    z1 = z1.reshape(z1.shape[0],1)
                    z2 = H @ y[:,[1]]
                    z2 = z2.reshape(z2.shape[0],1)
                    z = z1 - z2

                    # Solve the linear system for A and b
                    A = sp.sparse.hstack([H, z])
                    b = (beta*H_e1)

                    to_split = sp.sparse.linalg.lsqr(A, b.reshape(b.shape[0],1))[0]
                    
                    # the last element of to_split is the last element of gamma[i], the other elements are the elements of y[:[i]]
                    y[:,[i]] = to_split[:-1]
                    gamma[i] = to_split[-1]

                    # update x
                    x_new[:,i] = x[:,i] + V @ y[:,[i]]

                    # update res[i]
                    res[i] = (a[i]/a[k])*gamma[i]*res[k]

                else:
                    if res[i] < tau:
                        print("res[", i, "] is smaller than tau = ", tau, " at iteration ", iter)

        iter += 1
        x = x_new

    raise Exception('Maximum number of iterations reached')

Random Numerical Test Case on a small-world random graph

In [6]:
G = nx.watts_strogatz_graph(1000, 4, 0.1)
Pt, v = google_matrix_sparse(G)
# a = [0.85, 0.86, 0.87, 0.88, 0.89, 0.90, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99]
a = [0.85, 0.90, 0.95, 0.99]
n = len(G.nodes)
x = sp.sparse.random(n, len(a), density=0.1, format='lil')

In [7]:
x, iter, mv = Algo4(Pt, v, 100, a, 1e-6, 100, x)


iter =  1
res:  [0.00558049 0.00351364 0.00166436 0.00031942]
err =  0.005580489988532435
k =  0

iter =  2
res:  [ 6.30826211e-03  1.27202498e-07 -1.07175457e-08 -1.83681405e-08]
err =  0.006308262114154177
k =  0

iter =  3
res:  [ 6.30826211e-03  1.27202498e-07 -1.07175457e-08 -1.83681405e-08]
err =  0.006308262114154177
k =  0

iter =  4
res:  [ 6.30826211e-03  1.27202498e-07 -1.07175457e-08 -1.83681405e-08]
err =  0.006308262114154177
k =  0

iter =  5
res:  [ 6.30826211e-03  1.27202498e-07 -1.07175457e-08 -1.83681405e-08]
err =  0.006308262114154177
k =  0

iter =  6
res:  [ 6.30826211e-03  1.27202498e-07 -1.07175457e-08 -1.83681405e-08]
err =  0.006308262114154177
k =  0

iter =  7
res:  [ 6.30826211e-03  1.27202498e-07 -1.07175457e-08 -1.83681405e-08]
err =  0.006308262114154177
k =  0

iter =  8
res:  [ 6.30826211e-03  1.27202498e-07 -1.07175457e-08 -1.83681405e-08]
err =  0.006308262114154177
k =  0

iter =  9
res:  [ 6.30826211e-03  1.27202498e-07 -1.07175457e-08 -1.8368140

Exception: Maximum number of iterations reached